In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import odl
import odl.contrib.tensorflow
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
#from util import evaluate, display_progress, visualize

In [ ]:
BATCH_SIZE = 1
train_list = None 
eval_list = None

In [5]:
def evaluate():
    pass

def display_progress():
    pass

def visualize():
    pass

In [2]:
session = tf.InteractiveSession()

In [3]:
class orbitLoss(odl.solvers.Functional):
    def __init__(self, domain):
        super(orbitLoss, self).__init__(domain) 
        
    def localReg(self, prod_elem):
        return theta, fun, reco_rotated 


    def _call(self, prod_elem):
        return self.localReg(prod_elem)[1] #  optimal funtional value


    # For performance OrbitLossGradientOperator should maybe defined outside of orbitLoss?
    @property
    def gradient(self):
        class OrbitLossGradientOperator(odl.Operator):
            def __init__(self, domain, outer_instance):
                super(OrbitLossGradientOperator, self).__init__(domain=domain, range=domain)
                self.outer_instance = outer_instance
            def _call(self, prod_elem):
                x = prod_elem[0] #  Ground truth
                y = prod_elem[1] #  Reco
                grad_x = np.zeros_like(x) #  Lets set the dervative wrt ground truth-part to zero. It will not propagate back to network params.
                _, _, reco_rotated = self.outer_instance.localReg(prod_elem) #  
                grad_y = reco_rotated - x
                grad = self.domain.element([grad_x, grad_y])
                return grad
               
        return OrbitLossGradientOperator(self.domain, self)


In [5]:
def train_net(result_tensor):
    max_iter = 100
    for i in range(max_iter):
        ground_truth, data = get_next_batch(BATCH_SIZE) 

        session.run(optimizer, feed_dict={x_true: ground_truth, y: data})

        display_progress(i, max_iter, result_tensor, y, test_data, test_gt, every=20)

In [6]:
with tf.name_scope('placeholders'):
    x_true = tf.placeholder(tf.float64, shape=[None, 96, 96, 96, 1], name="x_true")
    y = tf.placeholder(tf.float64, shape=[None, 96, 96, 96, 1], name="y")

space = odl.ProductSpace(odl.uniform_discr(min_pt=[0]*3, max_pt=[1]*3, shape=[96]*3), 2)
loss_op = orbitLoss(domain=space)
tf_loss_op = odl.contrib.tensorflow.as_tensorflow_layer(loss_op, name='tf_loss_op')

In [7]:
with tf.name_scope('orbit_loss'):
    current = y

    for i in range(2):
        dx = tf.contrib.layers.conv3d(current, num_outputs=32, kernel_size=3)
        dx = tf.contrib.layers.conv3d(dx, num_outputs=32, kernel_size=3)
        dx = tf.contrib.layers.conv3d(dx, num_outputs=1, kernel_size=1,
                                      activation_fn=None)

        current = current + dx

    denoised = current

with tf.name_scope('optimizer_orbit_loss'):
    stack = tf.stack([x_true[0, :, :, 0], denoised[0, :, :, 0]])[None,..., None]

    loss = tf_loss_op(stack)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

tf.global_variables_initializer().run()

In [8]:
train_net(denoised)

'3000/3000 Quotient metric: 0.00463'

'3000/3000 L2 metric      : 0.10439'

In [ ]:
visualize()